# ItaData2024
modal example

In [1]:
using MLJ, ModalDecisionTrees
using SoleDecisionTreeInterface, Sole, SoleData
using CategoricalArrays
using DataFrames, JLD2, CSV
using Audio911
using Random

### Open .jld2 file
the file contains 504 samples of respiratory sound, labeled with 2 classes: healty and pneumonia

In [2]:
ds_path = "/datasets/respiratory_Healthy_Pneumonia"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

### Audio features extraction function
This function is called for every audio sample and extract 51 features:
26 bands of the mel spectrogram,
13 coefficients of the mfcc
12 spectral features: centroid, crest, entropy, f0, flatness, flux, kurtosis, rolloff, skewness, decrease, slope, spread

In [3]:
nan_replacer!(x::AbstractArray{Float64}) = replace!(x, NaN => 0.0)

function afe(x::AbstractVector{Float64}; get_only_melfreq=false)
    # -------------------------------- parameters -------------------------------- #
    # audio module
    sr = 8000
    norm = true
    speech_detection = false
    # stft module
    stft_length = 256
    win_type = (:hann, :periodic)
    win_length = 256
    overlap_length = 128
    stft_norm = :power                      # :power, :magnitude, :pow2mag
    # mel filterbank module
    nbands = 26
    scale = :mel_htk                        # :mel_htk, :mel_slaney, :erb, :bark
    melfb_norm = :bandwidth                 # :bandwidth, :area, :none
    freq_range = (300, round(Int, sr / 2))
    # mel spectrogram module
    db_scale = false
    # mfcc module
    ncoeffs = 13
    rectification = :log                    # :log, :cubic_root
    dither = true
    # f0 module
    method = :nfc
    f0_range = (50, 400)

    # --------------------------------- functions -------------------------------- #
    # audio module
    audio = load_audio(
        file=x,
        sr=sr,
        norm=norm,
    );

    stftspec = get_stft(
        audio=audio,
        stft_length=stft_length,
        win_type=win_type,
        win_length=win_length,
        overlap_length=overlap_length,
        norm=stft_norm
    );

    # mel filterbank module
    melfb = get_melfb(
        stft=stftspec,
        nbands=nbands,
        scale=scale,
        norm=melfb_norm,
        freq_range=freq_range
    );

    if get_only_melfreq
        return melfb.freq
    end

    # mel spectrogram module
    melspec =  get_melspec(
        stft=stftspec,
        fbank=melfb,
        db_scale=db_scale
    );

    # mfcc module
    mfcc = get_mfcc(
        source=melspec,
        ncoeffs=ncoeffs,
        rectification=rectification,
        dither=dither,
    );

    # f0 module
    f0 = get_f0(
        source=stftspec,
        method=method,
        freq_range=f0_range
    );

    # spectral features module
    spect = get_spectrals(
        source=stftspec,
        freq_range=freq_range
    );

    x_features = hcat(
        melspec.spec',
        mfcc.mfcc',
        f0.f0,
        spect.centroid,
        spect.crest,
        spect.entropy,
        spect.flatness,
        spect.flux,
        spect.kurtosis,
        spect.rolloff,
        spect.skewness,
        spect.decrease,
        spect.slope,
        spect.spread
    );

    nan_replacer!(x_features)

    return x_features
end

afe (generic function with 1 method)

### Compute DataFrame of features

In [4]:
color_code = Dict(:red => 31, :green => 32, :yellow => 33, :blue => 34, :magenta => 35, :cyan => 36)
freq = round.(Int, afe(x[1, :audio]; get_only_melfreq=true))
r_select = r"\e\[\d+m(.*?)\e\[0m"
variable_names = [
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:26]...,
    ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:13]...,
    "\e[$(color_code[:green])mf0\e[0m", "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
]

X = DataFrame([name => Vector{Float64}[] for name in [match(r_select, v)[1] for v in variable_names]])

features = [minimum, maximum]

for i in 1:nrow(x)
    push!(X, collect(eachcol(afe(x[i, :audio]))))
end

yc = CategoricalArray(y);

### Data compression for modal analysis

In [5]:
train_ratio = 0.8
train, test = partition(eachindex(yc), train_ratio, shuffle=true)
# train, test = partition(eachindex(yc), train_ratio, shuffle=false) ### Debug
X_train, y_train = X[train, :], yc[train]
X_test, y_test = X[test, :], yc[test]

println("Training set size: ", size(X_train), " - ", length(y_train))
println("Test set size: ", size(X_test), " - ", length(y_test))

Training set size: (403, 51) - 403
Test set size: (101, 51) - 101


### Train a model

In [6]:
learned_dt_tree = begin
    model = ModalDecisionTree(; relations = :IA7, features = features)
    mach = machine(model, X_train, y_train) |> fit!
end

report(learned_dt_tree).printmodel(variable_names_map=variable_names);

┌ Info: Precomputing logiset...
└ @ ModalDecisionTrees.MLJInterface /home/paso/.julia/dev/ModalDecisionTrees.jl/src/interfaces/MLJ/wrapdataset.jl:135
┌ Info: Training machine(ModalDecisionTree(max_depth = nothing, …), …).
└ @ MLJBase /home/paso/.julia/packages/MLJBase/7nGJF/src/machines.jl:499


▣ ⟨G⟩(min[mfcc3] < -0.49560830522534305)
├✔ ⟨G⟩((min[mfcc3] < -0.49560830522534305) ∧ ⟨A̅O̅⟩(min[cntrd] ≥ 1232.5140291243235))
│├✔ ⟨G⟩((min[mfcc3] < -0.49560830522534305) ∧ ⟨A̅O̅⟩((min[cntrd] ≥ 1232.5140291243235) ∧ (min[skwns] ≥ 0.6444504128467891)))
││├✔ Healthy
││└✘ ⟨G⟩((min[mfcc3] < -0.49560830522534305) ∧ ⟨A̅O̅⟩((min[cntrd] ≥ 1232.5140291243235) ∧ ⟨G⟩(min[mfcc7] < -1.2128523450308333)))
││ ├✔ Healthy
││ └✘ Pneumonia
│└✘ ⟨G⟩((min[mfcc3] < -0.49560830522534305) ∧ (min[mel10=1074Hz] < 2.3116095963813164e-7))
│ ├✔ Healthy
│ └✘ ⟨G⟩((min[mfcc3] < -0.49560830522534305) ∧ (min[mel11=1179Hz] < 7.3804430852378596e-6))
│  ├✔ ⟨G⟩((min[mfcc3] < -0.49560830522534305) ∧ (min[mel11=1179Hz] < 7.3804430852378596e-6) ∧ ⟨D̅B̅E̅⟩(max[mfcc9] < 0.1420268767345329))
│  │├✔ Pneumonia
│  │└✘ ⟨G⟩((min[mfcc3] < -0.49560830522534305) ∧ (min[mel11=1179Hz] < 7.3804430852378596e-6) ∧ (min[mel9=975Hz] < 7.1760123676389365e-6))
│  │ ├✔ ⟨G⟩((min[mfcc3] < -0.49560830522534305) ∧ (min[mel11=1179Hz] < 7.38044308523785

### Model inspection & rule study

In [7]:
_, mtree = report(mach).sprinkle(X_test, y_test)
sole_dt = ModalDecisionTrees.translate(mtree)

printmodel(sole_dt; show_metrics = true, variable_names_map=variable_names);

Applying tree... 100%|███████████████████████████████████| Time: 0:00:07


▣ ⟨G⟩(min[mfcc3] < -0.49560830522534305)
├✔ ⟨G⟩((min[mfcc3] < -0.49560830522534305) ∧ ⟨A̅O̅⟩(min[cntrd] ≥ 1232.5140291243235))
│├✔ ⟨G⟩((min[mfcc3] < -0.49560830522534305) ∧ ⟨A̅O̅⟩((min[cntrd] ≥ 1232.5140291243235) ∧ (min[skwns] ≥ 0.6444504128467891)))
││├✔ ⟨G⟩((min[mfcc3] < -0.49560830522534305) ∧ ⟨A̅O̅⟩((min[cntrd] ≥ 1232.5140291243235) ∧ (min[skwns] ≥ 0.6444504128467891) ∧ ⟨G⟩(min[mfcc3] ≥ 0.377997290238297)))
│││├✔ Healthy : (ninstances = 26, ncovered = 26, confidence = 0.88, lift = 1.0)
│││└✘ Healthy : (ninstances = 0, ncovered = 0, confidence = NaN, lift = NaN)
││└✘ ⟨G⟩((min[mfcc3] < -0.49560830522534305) ∧ ⟨A̅O̅⟩((min[cntrd] ≥ 1232.5140291243235) ∧ ⟨G⟩(min[mfcc7] < -1.2128523450308333)))
││ ├✔ Healthy : (ninstances = 2, ncovered = 2, confidence = 0.0, lift = NaN)
││ └✘ Pneumonia : (ninstances = 1, ncovered = 1, confidence = 0.0, lift = NaN)
│└✘ ⟨G⟩((min[mfcc3] < -0.49560830522534305) ∧ (min[mel10=1074Hz] < 2.3116095963813164e-7))
│ ├✔ Healthy : (ninstances = 8, ncovered = 8, conf

### Extract rules that are at least as good as a random baseline model

In [8]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0);
printmodel.(interesting_rules; show_metrics = true, variable_names_map=variable_names);

▣ ⟨G⟩((min[mfcc3] < -0.49560830522534305) ∧ ⟨A̅O̅⟩((min[cntrd] ≥ 1232.5140291243235) ∧ (min[skwns] ≥ 0.6444504128467891) ∧ ⟨G⟩(min[mfcc3] ≥ 0.377997290238297)))  ↣  Healthy : (ninstances = 101, ncovered = 26, coverage = 0.26, confidence = 0.88, lift = 1.86, natoms = 4)
▣ ⟨G⟩((min[mfcc3] < -0.49560830522534305) ∧ (min[mel10=1074Hz] < 2.3116095963813164e-7)) ∧ [G]((min[mfcc3] < -0.49560830522534305) → [A̅O̅](min[cntrd] < 1232.5140291243235))  ↣  Healthy : (ninstances = 101, ncovered = 8, coverage = 0.08, confidence = 0.62, lift = 1.32, natoms = 4)
▣ ⟨G⟩((min[mfcc3] < -0.49560830522534305) ∧ (min[mel11=1179Hz] < 7.3804430852378596e-6) ∧ ⟨D̅B̅E̅⟩((max[mfcc9] < 0.1420268767345329) ∧ (max[mel1=359Hz] ≥ 0.0075525705629675075))) ∧ [G]((min[mfcc3] < -0.49560830522534305) → [A̅O̅](min[cntrd] < 1232.5140291243235)) ∧ [G]((min[mfcc3] < -0.49560830522534305) → (min[mel10=1074Hz] ≥ 2.3116095963813164e-7))  ↣  Pneumonia : (ninstances = 101, ncovered = 1, coverage = 0.01, confidence = 1.0, lift = 1.91

### Simplify rules while extracting and prettify result

In [9]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0, normalize = true);
printmodel.(interesting_rules; show_metrics = true, syntaxstring_kwargs = (; threshold_digits = 2), variable_names_map=variable_names);

▣ ⟨G⟩((min[mfcc3] < -0.5) ∧ ⟨A̅O̅⟩((min[cntrd] ≥ 1232.51) ∧ (min[skwns] ≥ 0.64) ∧ ⟨G⟩(min[mfcc3] ≥ 0.38)))  ↣  Healthy : (ninstances = 101, ncovered = 26, coverage = 0.26, confidence = 0.88, lift = 1.86, natoms = 4)
▣ ⟨G⟩((min[mfcc3] < -0.5) ∧ (min[mel10=1074Hz] < 0.0)) ∧ [G]((min[mfcc3] < -0.5) → [A̅O̅](min[cntrd] < 1232.51))  ↣  Healthy : (ninstances = 101, ncovered = 8, coverage = 0.08, confidence = 0.62, lift = 1.32, natoms = 4)
▣ ⟨G⟩((min[mfcc3] < -0.5) ∧ (min[mel11=1179Hz] < 0.0) ∧ ⟨D̅B̅E̅⟩((max[mfcc9] < 0.14) ∧ (max[mel1=359Hz] ≥ 0.01))) ∧ [G]((min[mfcc3] < -0.5) → [A̅O̅](min[cntrd] < 1232.51)) ∧ [G]((min[mfcc3] < -0.5) → (min[mel10=1074Hz] ≥ 0.0))  ↣  Pneumonia : (ninstances = 101, ncovered = 1, coverage = 0.01, confidence = 1.0, lift = 1.91, natoms = 8)
▣ ⟨G⟩((min[mfcc3] < -0.5) ∧ (min[mel11=1179Hz] < 0.0) ∧ ⟨D̅B̅E̅⟩(max[mfcc9] < 0.14)) ∧ [G]((min[mfcc3] < -0.5) → [A̅O̅](min[cntrd] < 1232.51)) ∧ [G]((min[mfcc3] < -0.5) → (min[mel10=1074Hz] ≥ 0.0)) ∧ [G]((min[mfcc3] < -0.5) → (

### Directly access rule metrics

In [10]:
readmetrics.(listrules(sole_dt; min_lift=1.0, min_ninstances = 0))

9-element Vector{@NamedTuple{ninstances::Int64, ncovered::Int64, coverage::Float64, confidence::Float64, lift::Float64, natoms::Int64}}:
 (ninstances = 101, ncovered = 26, coverage = 0.25742574257425743, confidence = 0.8846153846153846, lift = 1.861378205128205, natoms = 4)
 (ninstances = 101, ncovered = 8, coverage = 0.07920792079207921, confidence = 0.625, lift = 1.3151041666666667, natoms = 4)
 (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 1.9056603773584906, natoms = 8)
 (ninstances = 101, ncovered = 11, coverage = 0.10891089108910891, confidence = 0.7272727272727273, lift = 1.385934819897084, natoms = 11)
 (ninstances = 101, ncovered = 4, coverage = 0.039603960396039604, confidence = 1.0, lift = 2.104166666666667, natoms = 13)
 (ninstances = 101, ncovered = 6, coverage = 0.0594059405940594, confidence = 0.8333333333333334, lift = 1.7534722222222223, natoms = 9)
 (ninstances = 101, ncovered = 2, coverage = 0.019801980198019802, confidenc

### Show rules with an additional metric (syntax height of the rule's antecedent)

In [11]:
printmodel.(sort(interesting_rules, by = readmetrics); show_metrics = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), variable_names_map=variable_names);

▣ ⟨G⟩((min[mfcc3] < -0.49560830522534305) ∧ (min[mel11=1179Hz] < 7.3804430852378596e-6) ∧ ⟨D̅B̅E̅⟩((max[mfcc9] < 0.1420268767345329) ∧ (max[mel1=359Hz] ≥ 0.0075525705629675075))) ∧ [G]((min[mfcc3] < -0.49560830522534305) → [A̅O̅](min[cntrd] < 1232.5140291243235)) ∧ [G]((min[mfcc3] < -0.49560830522534305) → (min[mel10=1074Hz] ≥ 2.3116095963813164e-7))  ↣  Pneumonia : (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 1.9056603773584906, natoms = 8, height = 7)
▣ ⟨G⟩(min[mel25=3491Hz] < 8.801223697933641e-7) ∧ [G](min[mfcc3] ≥ -0.49560830522534305) ∧ [G](min[mfcc2] ≥ 1.496891913461671) ∧ [G]((min[mel25=3491Hz] < 8.801223697933641e-7) → (max[entrp] < 0.667843307101543)) ∧ [G]((min[mel25=3491Hz] < 8.801223697933641e-7) → (min[mel3=488Hz] ≥ 9.210170581853491e-7))  ↣  Pneumonia : (ninstances = 101, ncovered = 2, coverage = 0.019801980198019802, confidence = 1.0, lift = 1.9056603773584906, natoms = 7, height = 5)
▣ ⟨G⟩((min[mfcc2] < 1.496891913461671) ∧

### Pretty table of rules and their metrics

In [12]:
metricstable(interesting_rules; variable_names_map=col_names, metrics_kwargs = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))))

┌───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬────────────┬──────────┬────────────┬────────────┬─────────┬────────┬────────┐
│                                                                                                                                                                                                                                                                                                                                                                      